# Single image analysis

* This notebook is for analysis of analysis of single images taken at, say, 1 sun conditions at open and short circuit for example, using the PL imaging setup
* Assumes pre processing has already been done
    
### Some instructions: 
* Please run each cell in order, following the instructions
    * Press cntrl+enter to run a cell
* When prompted enter information into the feilds and press enter
* If you go back and re run a cell, that's fine but you have to run every cell after that again, and in order

This might break if all these files are not there! If you want to edit this code definately go ahead, a bunch of the functions live in seperate .py files, BUT please make a copy!!!

In [8]:
from external_imports import *
from image_process import *

# Plot parameters
from cmcrameri import cm
import matplotlib
from matplotlib.colors import LogNorm
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['font.family'] = "Century Gothic"

### Core process: Finding the folders/sample names
* Enter datapath and save path
* You can comment out the approriate parts depending on if they are all the same bandgap

In [2]:
datapath = r"P:\DATA\PREPROCESSED\Ideality_study\seo_preprocessed\0h"
savepath = r"D:\test"
if not os.path.isdir(savepath):
    os.makedirs(savepath)

path_db = path_process(datapath)
used_filter = filtcalf
number_of_suns = 1

bandgaps = {}
for key in path_db.keys():
    if key.lower() != 'white':
#         Different bandgaps (comment/ uncomment as needed):
        
#         string = "What's the bandgap of "+key+" in eV?: "
#         bandgap = float(input(string))
#         bandgaps[key] = bandgap

#         Same bandgaps (comment/ uncomment as needed):
        bandgaps[key] = 1.6 # change this

### Core process: PLQEs

In [35]:
white_params_filepath = f"{datapath}/white_params.csv"  # defines location of white ref parameter file

# Flattened lists of parameters to loop over
datapaths = []
savepaths = []
filenames = []
white_filepaths = []
white_params_filepaths = []
corrections = []
flux_1suns = []

for key in path_db.keys():
    # Calculate spectral correction
    bandgap = bandgaps[key]
    correction = calculate_spectral_correction(bandgap, used_filter)

    for pix in path_db[key]: 
        white_filepath = f"{datapath}/{key}/{pix}/white.npy"
        
        # Find out what type of measurements were done: 
        for _,conditions,_ in os.walk(f"{datapath}/{key}/{pix}"):
            break
        for condition in conditions:
            npy_files = find_npy(f"{datapath}/{key}/{pix}/{condition}")
            # Only look at files with single images: 
            if len(npy_files) > 1:
                continue
            
            # Create folder structure 
            if not os.path.isdir(f"{savepath}/{key}/{pix}/PLQE_{condition}"):
                os.makedirs(f"{savepath}/{key}/{pix}/PLQE_{condition}")
            
            # Append to holders
            datapaths.append(f"{datapath}/{key}/{pix}/{condition}")
            savepaths.append(f"{savepath}/{key}/{pix}/PLQE_{condition}")
            filenames.append(npy_files[0])
            white_filepaths.append(white_filepath)
            white_params_filepaths.append(white_params_filepath)
            corrections.append(correction)
            flux_1suns.append(float(npy_files[0].split('_')[1])*number_of_suns)
            
            
def process_one_PLQEmap(datapath_i,savepath_i,filename,white_filepath,white_params_filepath,correction,flux_1sun):
    
    bias, num_sun, flux, PLQE = PLQEmap(datapath_i, filename, white_filepath, white_params_filepath, correction, flux_1sun)
    savename = f"{bias}_{num_sun}_{flux}_.npy"
    np.save(f"{savepath_i}/{savename}", PLQE)

for datapath_i,savepath_i,filename,white_filepath,white_params_filepath,correction,flux_1sun in zip(datapaths,savepaths,filenames,white_filepaths,white_params_filepaths,corrections,flux_1suns):
    process_one_PLQEmap(datapath_i,savepath_i,filename,white_filepath,white_params_filepath,correction,flux_1sun)

In [36]:
plqe_plotrange = (0.1,0.0001)

for key in path_db.keys():
    for pix in path_db[key]:
         for condition in conditions:
            npy_files = find_npy(f"{datapath}/{key}/{pix}/{condition}")
            if len(npy_files) > 1:
                continue

            if not os.path.isdir(f"{savepath}/Plots/PLQE"):
                os.makedirs(f"{savepath}/Plots/PLQE")
                
            filename = find_npy(f"{savepath}/{key}/{pix}/PLQE_{condition}")[0]


            fig, ax = plt.subplots()
            imarr = np.load(f"{savepath}/{key}/{pix}/PLQE_{condition}/{filename}")
            im = ax.imshow(imarr, cmap=cm.imola, norm=LogNorm(vmin=plqe_plotrange[0], vmax=plqe_plotrange[1]))

            cbar = plt.colorbar(im,pad=0.01,aspect=20)
            cbar.set_label('PLQE', rotation=270,labelpad=25)

            scalebar_size = 0.1 # cm
            scalebar_offset = 10 # pix

            scalebar_width_pix = 0.1*pixels_per_cm
            scalebar_height_pix = scalebar_width_pix/10
            scalbar_y_position = imarr.shape[0]-scalebar_offset-scalebar_height_pix

            text_x = scalebar_offset + scalebar_width_pix/2
            text_y = scalbar_y_position

            scalebar = patches.Rectangle((scalebar_offset,scalbar_y_position),scalebar_width_pix,scalebar_height_pix,linewidth=1,edgecolor='k',facecolor='white')
            ax.add_patch(scalebar)
            txt = plt.text(text_x,text_y,"1 mm", color = "white", ha='center', va='bottom', fontweight='bold')
            txt.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='k')])

            plt.axis('off')

            plt.savefig(f"{savepath}/Plots/PLQE/PLQE_{key}_{pix}_{condition}.png", dpi=300, bbox_inches='tight')
            plt.close('all')

### QFLS
* Produces maps of QFLS
* Requires only an OC image

In [37]:
for key in path_db.keys():
    for pix in path_db[key]: 
        
        # Calculate Voc_rad
        bandgap = bandgaps[key]
        voc_rad_0 = vocradf(bandgap)
        jsc_by_j0 = np.exp(sci.e*voc_rad_0/(sci.k*298))-1
        voc_rad = (sci.k*298/sci.e)*np.log((jsc_by_j0*number_of_suns)+1)

        for condition in conditions:
            npy_files = find_npy(f"{datapath}/{key}/{pix}/{condition}")
            if len(npy_files) > 1:
                continue
            if not os.path.isdir(f"{savepath}/{key}/{pix}/QFLS_{condition}"):
                os.makedirs(f"{savepath}/{key}/{pix}/QFLS_{condition}")

            filename = find_npy(f"{savepath}/{key}/{pix}/PLQE_{condition}")[0]
            PLQE = np.load(f"{savepath}/{key}/{pix}/PLQE_{condition}/{filename}")
            QFLS =  voc_rad +  (sci.k*298/sci.e)*np.log(PLQE)

            np.save(f"{savepath}/{key}/{pix}/QFLS_{condition}/{filename}", QFLS)

* Plots:

In [38]:
QFLS_plotrange = (1.18,1.21)

for key in path_db.keys():
    for pix in path_db[key]:
         for condition in ['oc']:#conditions:
            npy_files = find_npy(f"{datapath}/{key}/{pix}/{condition}")
            if len(npy_files) > 1:
                continue

            if not os.path.isdir(f"{savepath}/Plots/QFLS"):
                os.makedirs(f"{savepath}/Plots/QFLS")
                
            filename = find_npy(f"{savepath}/{key}/{pix}/QFLS_{condition}")[0]


            fig, ax = plt.subplots()
            imarr = np.load(f"{savepath}/{key}/{pix}/QFLS_{condition}/{filename}")
            im = ax.imshow(imarr, cmap=cm.batlow, vmin=QFLS_plotrange[0], vmax=QFLS_plotrange[1])

            cbar = plt.colorbar(im,pad=0.01,aspect=20)
            cbar.set_label('QFLS(eV)', rotation=270,labelpad=25)

            scalebar_size = 0.1 # cm
            scalebar_offset = 10 # pix

            scalebar_width_pix = 0.1*pixels_per_cm
            scalebar_height_pix = scalebar_width_pix/10
            scalbar_y_position = imarr.shape[0]-scalebar_offset-scalebar_height_pix

            text_x = scalebar_offset + scalebar_width_pix/2
            text_y = scalbar_y_position

            scalebar = patches.Rectangle((scalebar_offset,scalbar_y_position),scalebar_width_pix,scalebar_height_pix,linewidth=1,edgecolor='k',facecolor='white')
            ax.add_patch(scalebar)
            txt = plt.text(text_x,text_y,"1 mm", color = "white", ha='center', va='bottom', fontweight='bold')
            txt.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='k')])

            plt.axis('off')

            plt.savefig(f"{savepath}/Plots/QFLS/QFLS_{key}_{pix}_{condition}.png", dpi=300, bbox_inches='tight')
            plt.close('all')

### Collection efficiency
* Plots colection efficiency :
($\frac{PL_{oc} - PL_{sc}}{PL_{oc}}$)
* Draws from PLQE_oc and PLQE_sc folder, so that needs to be present

In [39]:
for key in path_db.keys():
    for pix in path_db[key]:

        path_i = f"{savepath}/{key}/{pix}"
        oc_filename = find_npy(f"{path_i}/PLQE_oc")[0]
        sc_filename = find_npy(f"{path_i}/PLQE_sc")[0]

        im_oc = np.load(f"{path_i}/PLQE_oc/{oc_filename}")
        im_sc = np.load(f"{path_i}/PLQE_sc/{sc_filename}")
        col_eff = (im_oc - im_sc) / im_oc
        if not os.path.isdir(f"{path_i}/Q_col"):
            os.makedirs(f"{path_i}/Q_col")
            filename = find_npy(f"{path_i}/PLQE_oc")[0]
        np.save(f"{path_i}/Q_col/{filename}", col_eff)

* Plots:

In [40]:
# Run this to save the images
for key in path_db.keys():
    for pix in path_db[key]:

        path_i = f"{savepath}/{key}/{pix}/Q_col"
        savepath_i = f"{savepath}/Plots/Q_col"
        if not os.path.isdir(savepath_i):
            os.makedirs(savepath_i)

        filename = find_npy(path_i)[0]
        fig, ax = plt.subplots()


        imarr = np.load(f"{path_i}/{filename}")
        im = ax.imshow(imarr*100, cmap=cm.hawaii, vmin=0, vmax=100)
        cbar = plt.colorbar(im,pad=0.01,aspect=20)
        cbar.set_label('Collection efficiency (%)', rotation=270,labelpad=25)

        scalebar_size = 0.1 # cm
        scalebar_offset = 10 # pix

        scalebar_width_pix = 0.1*pixels_per_cm
        scalebar_height_pix = scalebar_width_pix/10
        scalbar_y_position = imarr.shape[0]-scalebar_offset-scalebar_height_pix

        text_x = scalebar_offset + scalebar_width_pix/2
        text_y = scalbar_y_position

        scalebar = patches.Rectangle((scalebar_offset,scalbar_y_position),scalebar_width_pix,scalebar_height_pix,linewidth=1,edgecolor='k',facecolor='white')
        ax.add_patch(scalebar)
        txt = plt.text(text_x,text_y,"1 mm", color = "white", ha='center', va='bottom', fontweight='bold')
        txt.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='k')])

        plt.axis('off')

        plt.savefig(f"{savepath}/Plots/Q_col/Q_col_{key}_{pix}_{condition}.png", dpi=300,bbox_inches='tight')
        plt.close('all')
